In [179]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import svm

In [180]:
flowers = load_iris()

dir(flowers)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [181]:
df = pd.DataFrame(flowers.data, columns=flowers.feature_names)

df['flower'] = flowers.target
df['flower'] = df['flower'].apply(lambda x: flowers.target_names[x])

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [182]:
from sklearn.model_selection import train_test_split, cross_val_score

train_x, test_x, train_y, test_y = train_test_split(df.drop(columns=['flower']), df['flower'], test_size=.50)

In [183]:
model = svm.SVC(kernel='rbf', C=30, gamma='auto')
model.fit(train_x, train_y)


SVC(C=30, gamma='auto')

In [184]:
model.score(test_x, test_y)

0.96

In [185]:
cross_val_score(svm.SVC(kernel='rbf', C=20, gamma='auto'), flowers.data, flowers.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [186]:
cross_val_score(svm.SVC(kernel='rbf', C=10, gamma='auto'), flowers.data, flowers.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [187]:
cross_val_score(svm.SVC(kernel='linear', C=10, gamma='auto'), flowers.data, flowers.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [188]:
from sklearn.model_selection import GridSearchCV

In [189]:
clf = GridSearchCV(svm.SVC(gamma='auto'), { 
    'C': [x for x in range(50)],
    'kernel': ['rbf', 'linear']
}, cv=5, return_train_score=False)

In [190]:
clf.fit(train_x, train_y)

c:\Users\flyin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\flyin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\flyin\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "c:\Users\flyin\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx"

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                               15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
                               27, 28, 29, ...],
                         'kernel': ['rbf', 'linear']})

In [191]:
cv_results_df = pd.DataFrame(clf.cv_results_)

In [192]:
cv_results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.0007,2.447950e-04,0.0000,0.000000,0,rbf,"{'C': 0, 'kernel': 'rbf'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,100
1,0.0005,2.611745e-07,0.0000,0.000000,0,linear,"{'C': 0, 'kernel': 'linear'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,99
2,0.0011,3.739857e-04,0.0009,0.000200,1,rbf,"{'C': 1, 'kernel': 'rbf'}",1.0,0.933333,1.0,0.933333,0.933333,0.960000,0.03266,52
3,0.0012,2.450286e-04,0.0008,0.000245,1,linear,"{'C': 1, 'kernel': 'linear'}",1.0,1.000000,1.0,0.933333,0.933333,0.973333,0.03266,43
4,0.0012,2.449312e-04,0.0009,0.000200,2,rbf,"{'C': 2, 'kernel': 'rbf'}",1.0,1.000000,1.0,0.933333,0.933333,0.973333,0.03266,43


In [193]:
cv_results_df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,0,rbf,NaN
1,0,linear,NaN
2,1,rbf,0.960000
3,1,linear,0.973333
4,2,rbf,0.973333
...,...,...,...
95,47,linear,0.986667
96,48,rbf,0.960000
97,48,linear,0.986667
98,49,rbf,0.960000


In [194]:
clf.best_params_

{'C': 4, 'kernel': 'linear'}

In [195]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
    'C': [1, 10, 20],
    'kernel': ['rbf', 'linear']
}, cv=5, return_train_score=False, n_iter=2)

rs.fit(train_x, train_y)
pd.DataFrame(rs.cv_results_)[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,linear,0.986667
1,20,rbf,0.933333
